In [ ]:
import nltk
from nltk.tokenize import RegexpTokenizer
import pandas as pd
from IPython.display import display
import numpy as np
from google.colab import drive

drive.mount('/content/drive')

nltk.download('punkt')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

###Reading Data In

In [ ]:
df_full = pd.read_csv('/content/drive/MyDrive/cs173_emotions_data_set.csv')
df_full = df_full.replace(np.nan, '', regex=True)
display(df_full)

,Sadness Lexicons,Sadness Sentences,Joy Lexicons,Joy Sentences,Fear Lexicons,Fear Sentences,Anger Lexicons,Anger Sentences,Surprise Lexicons,Surprise Sentence,Disgust Lexicons,Disgust Sentences,Sadness + Joy Lexicons,Sadness + Joy Sentences,Fear + Anger Lexicons,Fear + Anger Sentences,Surprise + Disgust Lexicons,Surprise + Disgust Sentences,Sadness + Joy + Fear Lexicons,Sadness + Joy + Fear Sentences
0,"abduction, devastate",The devastating news of the child's abduction ...,"child, laughter, sunny",It was a sunny summer morning and the laughter...,shiver,As he walked in the dead of night he could hea...,reckless,While driving his family to a restaurant a car...,"startle, unexpected",She was startled unexpectedly as everyone spra...,trash,She had forgotten to take out the trash before...,"melancholy, pleasant","When I visited my old childhood home, I felt a...","fearful, harm, provoking, rage",Tom was fearful of his bully and remained doci...,"revolting, shock",He opened the package of his food delivery and...,"anxious, proud, sorrowful",The parents watched their son leave for colleg...
1,"wretched, wrongly",The wretched people have chosen wrongly.,"youth, zeal",The youth are filled with zeal.,"abandoned, yell, youth",The abandoned youth could be heard yelling.,"abolish, wrongful",We must abolish it for its wrongful acts.,yelp,He let out a yelp when his friend jumped out f...,abhorrent,The mysterious slime emitted an abhorrent smell.,"abysmal, advance","They have advanced to the next round, despite ...","animosity, agitation",His animosity was made worse by the repeated\...,"bewildered, appalling",She was bewildered by their appalling behavior.,"apologize, admirable, aggression, afraid",After seeing the group’s admirable performance...
2,"Stifle, Mausoleum","She was not fond of graveyards, let alone the\...","Fidelity, Praiseworthy",Ms. Smith taught the lesson to fidelity. The o...,"\r\nAbyss, Cripple","Leaning over the edge, I peered into the abyss...","Mock, Violence",Your actions make a mockery of ethics! I must ...,"Jolt, Shatter",The sound of my alarms jolts me awake. It shat...,"Unnatural, Plague, Grotesque","His illness left his skin unnatural, like plas...","Winning, Closure","Despite finally winning, I feel robbed of clos...","Abomination, Monstrosity, Affront","What you have created is an abomination, a bon...","Rave, Betray, Dreadfully",Enough of your dreadful raving... Did you forg...,"Endless, Destination, Retirement, Weight",So begins an endless journey with no destinati...
3,"abandon,abandoned,abandonment,abduction,aborti...",I’m feeling very anxious about going back to w...,"absolution, abundance,abundant,accolade,accomp...",The women’s soccer team was proud that they ac...,"abandon,abandoned,abandonment,abduction,abhor,...",I’m afraid we might get trapped in the cave si...,"abandoned,abandonment,abhor,abhorrent,abolish,...","After getting yelled at for 30 minutes, Sam be...","underestimate,undiscovered,unexpected,unexpect...",I underestimated how urgent the call was.,"aberration,abhor,abhorrent,abject,abnormal,abo...",I can’t believe how wasteful the food industry...,"true,twinkle,unbeaten,unconstrained,undying,un...",Although my heart aches because I will never b...,"vermin,versus,veto,vicious,victim,victimized,v...",I hate Tony for wrongfully accusing me of stea...,"underestimate,undiscovered,unexpected,unexpect...",When we looked under the couch we found the un...,"wail,wallow,wan,wane,wanting,warfare,warp,wast...",She starts to weep because she knows that I mi...
4,lonely,Sometimes I feel like something is wrong with ...,"engaged, rekindle",It's crazy to see that Jane and Mike were able...,"onfined, tumor, scare",Ever since the doctors found a tumor in Jessie...,"grudge, incompetent",I cannot believe that you’re still holding a g...,"celebrity, aspiration",It’s always been an aspiration of mine to be a...,nasty,I think it’s so nasty that he’s a grown man an...,"tough, closure, glad",It must have been tough 

###Tokenization
Pre-processed through setting whole string to lower case, tokenizing by word, and removing punctuation

In [ ]:
#gets all column names as a list
def get_column_names(df):
  column_names = list(df.columns.values.tolist())
  return column_names

#returns all columns that contain sentences in a list
def get_sentence_column_names(col_names):
  sentence_col_names = []
  for i in range(len(col_names)):
    if i % 2 == 1:
      sentence_col_names.append(col_names[i])

  return sentence_col_names

#returns a data frame containing only sentence columns
def get_only_sentence_columns(df, sentence_col_names):
  sentence_columns = pd.DataFrame()

  for i in range(len(sentence_col_names)):
    sentence_columns[sentence_col_names[i]] = df[sentence_col_names[i]]
  
  return sentence_columns

#helper function that tokenizes a sentence and is used by tokenize_sentences
def tokenize_sentences_helper(sentence):
  sentence = sentence.lower()
  tokenizer = RegexpTokenizer(r'[^.,"\(\)\s]+')
  tokenized_sentence = tokenizer.tokenize(sentence)

  return tokenized_sentence

#tokenizes all cells of a data frame containing only sentences
def tokenize_sentences(sentences_df, col_names):
  for col in col_names:
    sentences_df[col] = sentences_df[col].apply(tokenize_sentences_helper)
  
  return sentences_df

column_names = get_column_names(df_full)
sentence_col_names = get_sentence_column_names(column_names)
all_sentences = get_only_sentence_columns(df_full,sentence_col_names)

tokenized_sentences_df = tokenize_sentences(all_sentences, sentence_col_names)
display(tokenized_sentences_df)

  

,Sadness Sentences,Joy Sentences,Fear Sentences,Anger Sentences,Surprise Sentence,Disgust Sentences,Sadness + Joy Sentences,Fear + Anger Sentences,Surprise + Disgust Sentences,Sadness + Joy + Fear Sentences
0,"[the, devastating, news, of, the, child's, abd...","[it, was, a, sunny, summer, morning, and, the,...","[as, he, walked, in, the, dead, of, night, he,...","[while, driving, his, family, to, a, restauran...","[she, was, startled, unexpectedly, as, everyon...","[she, had, forgotten, to, take, out, the, tras...","[when, i, visited, my, old, childhood, home, i...","[tom, was, fearful, of, his, bully, and, remai...","[he, opened, the, package, of, his, food, deli...","[the, parents, watched, their, son, leave, for..."
1,"[the, wretched, people, have, chosen, wrongly]","[the, youth, are, filled, with, zeal]","[the, abandoned, youth, could, be, heard, yell...","[we, must, abolish, it, for, its, wrongful, acts]","[he, let, out, a, yelp, when, his, friend, jum...","[the, mysterious, slime, emitted, an, abhorren...","[they, have, advanced, to, the, next, round, d...","[his, animosity, was, made, worse, by, the, re...","[she, was, bewildered, by, their, appalling, b...","[after, seeing, the, group’s, admirable, perfo..."
2,"[she, was, not, fond, of, graveyards, let, alo...","[ms, smith, taught, the, lesson, to, fidelity,...","[leaning, over, the, edge, i, peered, into, th...","[your, actions, make, a, mockery, of, ethics!,...","[the, sound, of, my, alarms, jolts, me, awake,...","[his, illness, left, his, skin, unnatural, lik...","[despite, finally, winning, i, feel, robbed, o...","[what, you, have, created, is, an, abomination...","[enough, of, your, dreadful, raving, did, you,...","[so, begins, an, endless, journey, with, no, d..."
3,"[i’m, feeling, very, anxious, about, going, ba...","[the, women’s, soccer, team, was, proud, that,...","[i’m, afraid, we, might, get, trapped, in, the...","[after, getting, yelled, at, for, 30, minutes,...","[i, underestimated, how, urgent, the, call, was]","[i, can’t, believe, how, wasteful, the, food, ...","[although, my, heart, aches, because, i, will,...","[i, hate, tony, for, wrongfully, accusing, me,...","[when, we, looked, under, the, couch, we, foun...","[she, starts, to, weep, because, she, knows, t..."
4,"[sometimes, i, feel, like, something, is, wron...","[it's, crazy, to, see, that, jane, and, mike, ...","[ever, since, the, doctors, found, a, tumor, i...","[i, cannot, believe, that, you’re, still, hold...","[it’s, always, been, an, aspiration, of, mine,...","[i, think, it’s, so, nasty, that, he’s, a, gro...","[it, must, have, been, tough, to, find, out, a...","[it’s, insane, how, daniel, was, able, to, act...","[it’s, concerning, how, pearson, isn’t, in, mu...","[it, really, makes, you, wonder, what, people,..."
5,"[the, abandoned, child, looked, up, with, sad,...","[the, clerk, asked, the, child, for, a, small,...","[the, unnerving, noises, and, echoes, coming, ...","[the, child’s, outburst, left, everyone, shock...","[my, heart, beat, with, joy, when, i, unexpect...","[the, awful, smell, coming, from, the, animal,...","[the, survivors, of, the, holocaust, carry, wi...","[when, the, officer, pulled, him, over, he, kn...","[i, felt, appalled, and, ashamed, when, there,...","[i, regret, not, taking, the, job, opportunity..."
6,"[the, death, of, her, dog, left, her, in, deep...","[after, months, of, hard, work, the, team, was...","[he, was, afraid, of, heights, and, refused, t...","[she, was, so, angry, when, she, found, out, t...","[the, unexpected, twist, at, the, end, of, the...","[the, smell, of, rotten, food, was, so, unplea...","[although, he, was, deeply, ashamed, of, his, ...","[watching, the, opponent, score, the, tying, p...","[the, awful, smell, from, the, dumpster, was, ...","[the, delightful, secluded, garden, made, her,..."
7,"[as, my, hand, left, the, piece, i, felt, crus...","[i, delighted, in, the, unconstrained, serenit...","[all, hope, disappears, as, they, look, aghast..

###Splitting Data into Train, Test, Validation

In [ ]:
training_data = tokenized_sentences_df.iloc[:30]
valid_data = tokenized_sentences_df.iloc[30:40]
test_data = tokenized_sentences_df.iloc[40:]

print("TRAININ DATA")
# print(training_data)
display(training_data)

print("\n\nVALIDATION DATA\n\n")
# print(valid_data)
display(valid_data)

print("\n\nTEST DATA\n\n")
# print(test_data)
display(test_data)

TRAININ DATA


,Sadness Sentences,Joy Sentences,Fear Sentences,Anger Sentences,Surprise Sentence,Disgust Sentences,Sadness + Joy Sentences,Fear + Anger Sentences,Surprise + Disgust Sentences,Sadness + Joy + Fear Sentences
0,"[the, devastating, news, of, the, child's, abd...","[it, was, a, sunny, summer, morning, and, the,...","[as, he, walked, in, the, dead, of, night, he,...","[while, driving, his, family, to, a, restauran...","[she, was, startled, unexpectedly, as, everyon...","[she, had, forgotten, to, take, out, the, tras...","[when, i, visited, my, old, childhood, home, i...","[tom, was, fearful, of, his, bully, and, remai...","[he, opened, the, package, of, his, food, deli...","[the, parents, watched, their, son, leave, for..."
1,"[the, wretched, people, have, chosen, wrongly]","[the, youth, are, filled, with, zeal]","[the, abandoned, youth, could, be, heard, yell...","[we, must, abolish, it, for, its, wrongful, acts]","[he, let, out, a, yelp, when, his, friend, jum...","[the, mysterious, slime, emitted, an, abhorren...","[they, have, advanced, to, the, next, round, d...","[his, animosity, was, made, worse, by, the, re...","[she, was, bewildered, by, their, appalling, b...","[after, seeing, the, group’s, admirable, perfo..."
2,"[she, was, not, fond, of, graveyards, let, alo...","[ms, smith, taught, the, lesson, to, fidelity,...","[leaning, over, the, edge, i, peered, into, th...","[your, actions, make, a, mockery, of, ethics!,...","[the, sound, of, my, alarms, jolts, me, awake,...","[his, illness, left, his, skin, unnatural, lik...","[despite, finally, winning, i, feel, robbed, o...","[what, you, have, created, is, an, abomination...","[enough, of, your, dreadful, raving, did, you,...","[so, begins, an, endless, journey, with, no, d..."
3,"[i’m, feeling, very, anxious, about, going, ba...","[the, women’s, soccer, team, was, proud, that,...","[i’m, afraid, we, might, get, trapped, in, the...","[after, getting, yelled, at, for, 30, minutes,...","[i, underestimated, how, urgent, the, call, was]","[i, can’t, believe, how, wasteful, the, food, ...","[although, my, heart, aches, because, i, will,...","[i, hate, tony, for, wrongfully, accusing, me,...","[when, we, looked, under, the, couch, we, foun...","[she, starts, to, weep, because, she, knows, t..."
4,"[sometimes, i, feel, like, something, is, wron...","[it's, crazy, to, see, that, jane, and, mike, ...","[ever, since, the, doctors, found, a, tumor, i...","[i, cannot, believe, that, you’re, still, hold...","[it’s, always, been, an, aspiration, of, mine,...","[i, think, it’s, so, nasty, that, he’s, a, gro...","[it, must, have, been, tough, to, find, out, a...","[it’s, insane, how, daniel, was, able, to, act...","[it’s, concerning, how, pearson, isn’t, in, mu...","[it, really, makes, you, wonder, what, people,..."
5,"[the, abandoned, child, looked, up, with, sad,...","[the, clerk, asked, the, child, for, a, small,...","[the, unnerving, noises, and, echoes, coming, ...","[the, child’s, outburst, left, everyone, shock...","[my, heart, beat, with, joy, when, i, unexpect...","[the, awful, smell, coming, from, the, animal,...","[the, survivors, of, the, holocaust, carry, wi...","[when, the, officer, pulled, him, over, he, kn...","[i, felt, appalled, and, ashamed, when, there,...","[i, regret, not, taking, the, job, opportunity..."
6,"[the, death, of, her, dog, left, her, in, deep...","[after, months, of, hard, work, the, team, was...","[he, was, afraid, of, heights, and, refused, t...","[she, was, so, angry, when, she, found, out, t...","[the, unexpected, twist, at, the, end, of, the...","[the, smell, of, rotten, food, was, so, unplea...","[although, he, was, deeply, ashamed, of, his, ...","[watching, the, opponent, score, the, tying, p...","[the, awful, smell, from, the, dumpster, was, ...","[the, delightful, secluded, garden, made, her,..."
7,"[as, my, hand, left, the, piece, i, felt, crus...","[i, delighted, in, the, unconstrained, serenit...","[all, hope, disappears, as, they, look, aghast..



VALIDATION DATA




,Sadness Sentences,Joy Sentences,Fear Sentences,Anger Sentences,Surprise Sentence,Disgust Sentences,Sadness + Joy Sentences,Fear + Anger Sentences,Surprise + Disgust Sentences,Sadness + Joy + Fear Sentences
30,"[the, lonely, and, hungry, puppy, whined, as, ...","[her, heart, melted, in, joy, as, she, watched...","[she, trembled, in, fear, as, she, stumbled, u...","[he, clenched, his, fists, and, teeth, holding...","[she, let, out, a, yelp, in, surprise, when, h...","[realizing, the, bread, had, an, abnormal, tex...","[at, a, young, age, she, was, disowned, and, t...","[molly, had, an, argument, with, her, parents,...","[sarah, came, back, from, a, month-long, vacat...","[coming, from, a, strict, family, tom’s, paren..."
31,"[whenever, alexa, remembered, how, the, love, ...","[ethan, was, elated, to, find, out, about, the...","[russia’s, attempted, conquest, of, ukraine, h...","[when, george, found, out, that, olivia, was, ...","[michael, was, astonished, to, see, harper’s, ...","[charlie, was, revolted, when, he, discovered,...","[while, liam, was, despondent, to, see, the, o...","[charlotte’s, terror, of, noah’s, harrowing, d...","[claire, could, not, hide, her, astonishment, ...","[while, jack, could, only, feel, despair, and,..."
32,"[as, i, stared, into, the, lifeless, eyes, of,...","[while, the, horde, of, zombies, began, surrou...","[as, the, thundering, steps, from, the, other,...","[the, scorching, sound, of, the, neighbors, ma...","[i, caught, myself, staggering, my, words, try...","[the, neglected, little, girl, was, kept, in, ...","[the, victim, was, such, a, suffering, sight, ...","[i, quickly, ran, and, yelled, at, the, horrid...","[the, veracity, of, him, to, even, think, abou...","[i, immediately, volunteered, with, glee, as, ..."
33,"[gas, became, overpriced, during, the, pandemic]","[the, mother, was, thankful, for, the, gift, f...","[the, siren, warned, the, residents, of, the, ...","[the, lack, of, whipped, cream, on, the, woman...","[when, i, was, stabbed, my, appendix, ruptured...","[the, parents, found, the, vulgar, language, o...","[the, man’s, coworkers, treated, him, to, a, d...","[the, evil, tyrant’s, army, pillaged, the, town]","[the, celebrity, is, under, investigation, for...","[sitting, across, from, his, date, after, the,..."
34,"[the, sickening, realization, finally, dawned,...","[it, was, finally, match, point, after, a, thr...","[when, i, walked, into, the, dimly, lit, aband...","[“you, really, think, i, would, trust, you, ag...","[ethan, looked, down, in, bewilderment, at, th...","[“no, way, you’re, going, to, drink, that, ”, ...","[tonight, was, the, night, of, my, big, perfor...","[“please, don’t, do, it, you’re, going, to, re...","[leon, my, ex-husband, pitifully, knelt, befor...","[i, looked, across, at, my, brother, jack, for..."
35,"[rohit:, whenever, i, think, about, the, times...","[rohit:, the, beach, has, always, been, a, hap...","[rohit:, getting, vaccinated, and, practicing,...","[rohit:, the, protesting, crowd, was, riled, u...","[rohit:, the, baseball, game, was, placed, on,...","[rohit:, the, smell, of, puke, and, from, the,...","[although, she, was, haunted, by, her, past, w...","[rohit:, as, he, slammed, the, door, shut, beh...","[i, cringed, at, the, sight, of, the, dismissa...","[rohit:, i, often, times, are, reminded, of, t..."
36,"[the, young, boy, sat, alone, on, the, curb, w...","[after, numerous, interviews, and, coding, cha...","[the, man's, heart, raced, as, he, walked, tow...","[the, boy, felt, the, fury, rise, within, him,...","[as, she, walked, into, the, darkened, room, s...","[the, taste, of, the, rotten, and, moldy, sand...","[as, he, watched, his, daughter, sink, the, wi...","[the, room, was, silent, except, for, the, sou...","[i, heard, a, loud, bang, and, rushed, outside...","[john, was, over, the, moon, when, he, receive..."
37,"[the, sorrowful, expression, upon, the, dying,...","[an, overwhelming, feeling, of, ecstasy, washe...","[his, heart, twitched, wit



TEST DATA




,Sadness Sentences,Joy Sentences,Fear Sentences,Anger Sentences,Surprise Sentence,Disgust Sentences,Sadness + Joy Sentences,Fear + Anger Sentences,Surprise + Disgust Sentences,Sadness + Joy + Fear Sentences
40,"[he, sat, alone, by, the, window, lost, in, a,...","[the, elegant, party, was, filled, with, exqui...","[the, student’s, hands, shook, nervously, as, ...","[she, couldn’t, help, but, feel, anger, bubbli...","[the, sudden, deluge, of, rain, caught, everyo...","[the, sight, of, the, massive, anaconda, slith...","[the, artist, felt, a, strange, mix, of, sadne...","[his, fear, of, losing, her, friendship, mixed...","[she, was, taken, aback, with, surprise, and, ...","[as, the, shepherd, watches, the, young, lamb,..."
41,"[tim, was, unlucky, he, had, a, traumatic, exp...","[john, achieved, his, lifelong, goal, of, gett...","[for, the, first, time, ryan, felt, fear, ther...","[herlock, shouted, at, the, prisoner, “32, sta...","[the, sudden, sneeze, from, henry, startled, h...","[that, dastardly, man, went, into, the, house,...","[we, commemorated, the, closure, of, our, stor...","[the, abandonment, of, those, children, are, a...","[i, saw, the, faces, of, those, who, were, att...","[we, were, about, to, reach, our, final, desti..."
42,"[after, his, wife, left, him, he, lost, his, j...","[the, music, that, was, playing, in, the, cafe...","[friday, the, thirteenth, always, elicited, an...","[hearing, the, various, hoaxes, that, his, fri...","[the, sudden, divorce, papers, from, her, husb...","[the, boy, was, so, interested, in, rats, that...","[the, sonnet, the, man, made, about, his, moth...","[the, hopelessness, that, he, felt, from, his,...","[the, scream, the, girls, let, out, from, the,...","[the, outburst, from, the, woman, made, all, o..."
43,"[the, news, caused, my, world, to, collapse, l...","[the, moment, i, laid, my, eyes, on, the, acce...","[when, the, thought, of, needing, an, abortion...","[filled, with, rage, her, father, stood, up, a...","[her, boyfriend’s, abrupt, breakup, without, a...","[jason, felt, disgusted, and, recoiled, at, th...","[we, felt, melancholic, as, we, watched, the, ...","[walking, down, the, dark, alley, he, noticed,...","[the, first, time, i, drank, alcohol, the, pun...","[link’s, quest, to, save, hyrule, from, calami..."
44,"[he, was, distraught, by, the, horrors, he, sa...","[pleased, with, her, performance, maggie, was,...","[he, felt, an, overwhelming, sense, of, hopele...","[offended, by, his, actions, kyle, had, to, re...","[the, wizard, vanished, before, the, child’s, ...","[jayden, thought, he, was, going, to, vomit, a...","[though, sandra, was, thrilled, to, be, finall...","[the, man, whimpered, after, violently, gettin...","[mark, spent, hours, trying, to, find, the, ro...","[though, he, was, rejoicing, that, the, stab, ..."
45,"[grief, i’ve, learned, is, really, just, love,...","[the, pride, of, their, accomplishment, welled...","[the, asylum, was, supposed, to, be, abandoned...","[everyday, i, wake, up, and, i, hope, you're, ...","[as, i, whipped, my, head, towards, the, sound...","[was, it, not, your, sin, that, trapped, the, ...","[her, heart, brimmed, so, fully, with, hopes, ...","[i, stifled, my, cries, and, clamped, my, hand...","[the, anticipation, of, sweetness, made, the, ...","[i, worry, that, someday, i’ll, forget, the, l..."
46,"[she, abandoned, me, last, night, after, sayin...","[i, was, grateful, to, have, met, him, his, yo...","[i, didn't, think, much, of, others', comments...","[i, started, yelling, the, most, abhorrent, th...","[she, yelped, when, she, accidentally, ran, in...","[i, abhor, him, so, much, that, staring, at, h...","[the, abandonment, she, had, went, through, ma...","[he, started, yelling, aggressively, to, fight...","[i, yelped, when, i, was, suddenly, abducted, ...","[she, yelled, urgently, before, i, was, able, ..."
47,"[richard, lowered, his, head, in, disbelief, a...","[the, father, looked, at, his, son’s, youthful...","[due, to, the, pouring, 

###Implementing Naive Bayes Classifier

In [ ]:
def calculate_priors(training_set, sentence_col_names):
  training_set_priors = dict()
  total_sentences = 0
  sanity_check = 0

  for col in sentence_col_names:
    col_values = list(training_set[col])

    for sentence in col_values:
      if len(sentence) != 0:
        
        total_sentences += 1

        if col not in training_set_priors:
          training_set_priors[col] = 1
        else:
          training_set_priors[col] += 1
  
  for col in sentence_col_names:
    training_set_priors[col] /= total_sentences
    sanity_check += training_set_priors[col]

  print(sanity_check) #check to see if adding all class probabilities together yields 100%

  return training_set_priors  
    
training_priors = calculate_priors(training_data, sentence_col_names)
print(training_priors)


1.0
{'Sadness Sentences': 0.10135135135135136, 'Joy Sentences': 0.10135135135135136, 'Fear Sentences': 0.10135135135135136, 'Anger Sentences': 0.10135135135135136, 'Surprise Sentence': 0.09797297297297297, 'Disgust Sentences': 0.10135135135135136, 'Sadness + Joy Sentences': 0.10135135135135136, 'Fear + Anger Sentences': 0.10135135135135136, 'Surprise + Disgust Sentences': 0.0945945945945946, 'Sadness + Joy + Fear Sentences': 0.09797297297297297}
